In [ ]:
#this creates a csv file 'universityList.csv' containing name, key and link for all university.
# Note to self
# ------- Do not Execute this cell -----------

getUniversityList()

In [1]:
import requests
from lxml import html
import sys
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import re
import urllib.request
from requests.exceptions import HTTPError
import pandas as pd
import numpy as np
import time
import random

In [ ]:
def open_url_2_soup(url, session_request = None):
    """
    open webpage and convert to beautiful soup object
    returns: page_soup(soup object)
    params: url(str) - url of the page
    
    """
    proxies = [{
        "http": 'http://14.102.81.195:21776', 
        "https": 'http://14.102.81.195:21776'
    }]
    
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'}
    if(session_request == None):
        page = requests.get(url, headers = headers,verify = False, allow_redirects = False)
    else:
        page = session_request.get(url, headers = headers,verify = False, allow_redirects = False)
    if(page.status_code == 200):
        page_soup = soup(page.content,'lxml')
        return page_soup
    elif(page.status_code == 302):
        print("error code : ",page.status_code)
        print(url)
        time.sleep(900)
        token,session_req = getAuthSession()
        open_url_2_soup(url, session_req)
    else:
        print(page.status_code)
        raise HTTPError()
        
        

In [ ]:
# run it once you open the notebook
def getAuthSession():
    """
    generate session with login credentials
    params: None
    returns: token(string) - the csrf token value
             session_requests(session object): session with login posted
    """
    USERNAME = "webscrapingrb@gmail.com"
    PASSWORD = "priyabashyam"
    LOGIN_URL = "https://yocket.in/account/login"
    session_requests = requests.session()
    headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}
    result = session_requests.get(LOGIN_URL, verify = False, headers = headers)
    tree = html.fromstring(result.text)
    authenticity_token = list(set(tree.xpath("//input[@name='_csrfToken']/@value")))[0]
    payload = {
        "email": USERNAME,
         "password": PASSWORD,
         "_csrfToken": authenticity_token
    }

    # Perform login
    login_req = session_requests.post(LOGIN_URL, data=payload, headers=dict(referer=LOGIN_URL))
    return (authenticity_token,session_requests)

In [ ]:
def getUniversityLookup(url):
    """
    get the list of university and it's basic info from ONE page
    returns : univ_df - dataframe containing name,key and url of universities one page at a time
    params: url - url of the page
    
    """
    univ_name = []
    univ_link = []
    univ_soup = open_url_2_soup(url)
    univ_detail_tag = univ_soup.findAll("div",{"class":"col-sm-9 col-xs-12"})
    for tag in univ_detail_tag:
        a = tag.find("a",href=True)
        univ_name.append(a.text)
        univ_link.append("https://yocket.in"+a['href'])
    #data cleaning 
    univ_df = pd.DataFrame({"name":np.asarray(univ_name),"href": np.asarray(univ_link)})
    print(univ_df['name'].str.split('(',n = 1, expand = True))
    univ_df[['name','keys']] = univ_df['name'].str.split('(',n = 1, expand = True)
    univ_df['keys'] = univ_df['keys'].apply(lambda x: x[:-1])
    return univ_df

In [ ]:
def getUniversityList():
    """
    creates a csv file of all university detail
    returns: UniversityList_df - dataframe containg the basic details of top 500 engineering schools
    params: None
    
    """
    UniversityList_df = pd.DataFrame()
    count = 1
    while(count <= 16):
        url = "https://yocket.in/universities?page=" + str(count)
        temp_df = getUniversityLookup(url)
#         print(temp_df)
        UniversityList_df = UniversityList_df.append(temp_df,ignore_index=True)
#         print(UniversityList_df)
        time.sleep(5) 
        count += 1
    UniversityList_df.to_csv('universityList.csv', encoding='utf-8')
    return UniversityList_df


In [ ]:

def getUniversityDetail(url):
    """
    get the details of a single university
    params: url(string) - url of the site
    returns : univ_detail(dict) - dictionary containing details of a single university
    
    """
    univ_detail = {}
    page_soup = open_url_2_soup(url)
    univ_tag = page_soup.findAll("div",{"class":"col-sm-3 col-xs-6"})
    univ_detail['name'] = page_soup.findAll("h1")[0].text
    tag_no = 1;
    for tag in univ_tag:
        if(tag_no in [1,2,3,4,6,8,9]):
            name,data = extractUnivDetails(tag,tag_no)
            univ_detail[name] = data
        tag_no+= 1
    try:
        univ_detail['admitList'] = univ_detail['student_link']+"2"
        univ_detail['rejectList'] = univ_detail['student_link']+"3"
        univ_detail['appliedList'] = univ_detail['student_link']+"1"
        del univ_detail['student_link']
    except:
        univ_detail['admitList'] = None
        univ_detail['rejectList'] = None
        univ_detail['appliedList'] = None
        del univ_detail['student_link']
    return(univ_detail)
# getUniversityDetail("https://yocket.in/university-reviews/california-state-university-chico-3145/engineering")

In [ ]:
def extractUnivDetails(tag,tag_no):
    """
    returns : univ_dict(dictionary) - dictionary containing university(single) detail
    params: tag(soup element) - the div containing the element
            tag_no(int) - number of div
    """
    univ_dict = {}
    if(tag_no in [1,2,8,9]):
        return(tag.h3.small.text,(tag.br.previous_sibling)[1:])
    elif(tag_no ==3):
        return (tag.h3.findAll("small")[1].text,tag.h3.find("small").previous_sibling)
    elif(tag_no ==4):
        return (tag.h3.findAll("small")[1].text,(tag.h3.find("small").previous_sibling)[2:])
    elif(tag_no == 6):
        try:
            a = tag.findAll("a",href=True)[0]
        except:
            return("student_link",None)
        else:
            return("student_link",("https://yocket.in"+a['href'])[:-1])


In [ ]:
def univDetail_to_csv():
    """
    generate csv file containing university details
    params : None
    returns : None
    """
    univ_list = pd.read_csv('universityList.csv')
    universityDetail = pd.DataFrame()
    counter = 51
    while(counter <= 100):
        link = univ_list.iloc[counter]['href']
        dict_temp = getUniversityDetail(link)
        universityDetail = universityDetail.append(dict_temp,ignore_index = True)
        print(counter)
        counter += 1
        time.sleep(10)
#     print(universityDetail)
    universityDetail.to_csv('universityDetailList.csv', encoding='utf-8',header = False,mode = 'a')

In [ ]:
#create the csv file of university list with all it's parameter
#  Note to self
# -----------DO NOT EXECUTE THIS CELL --------------
univDetail_to_csv()   

In [ ]:
def getStdList(page_soup,admit_status):
    """
    calculates the details of user for a single page
    params : admit_status(string) - admit/reject string
             page_soup(soup object) - soup of each page of student list of a university
    returns: dataframe containing user detail from a single page
    """
    name_tag = page_soup.findAll("input" , {"id" : "users-view-search-universities"})[0]
    name = name_tag['value']
    grading = page_soup.findAll("div",{"class" : "col-sm-6"})[1:]
#     print(grading[0])
    student_df = pd.DataFrame()
    for tag in grading:
#         print(tag)
        std_data = {}
        std_data['college'] = name
        std_data['admit/reject'] = admit_status
        a = tag.findAll("a",href = True)[0]
#         print(a)
        std_data['Name'] = a.text #Tag for name of student
#         std_data['studentLink'] - a['href']
        std_data['CourseName'] = tag.find("small").text #Tag for course
        student_data_tag = tag.findAll("div",{"class":"col-sm-3 col-xs-6"})
        for data_tag in student_data_tag:
            heading = data_tag.find("strong").text
            data = data_tag.findNext("br").next_sibling
            if (heading == "GRE"):
                std_data['GRE'] = data.string

            elif(heading == "TOEFL" or heading == "IELTS" or heading == "ENG TEST"):
                std_data['ENG_TEST'] = data.string

            elif(heading == "UNDERGRAD"):
                std_data['CGPA']= data.string

            elif(heading == "WORK EX"):
                std_data['WorkExp'] = data.string
#         print(std_data)
        student_df = student_df.append(std_data, ignore_index=True)
#     print(student_df)
    return student_df


In [ ]:
def getStudentDetailList(url,session_request = None, admit = None):
    """
    gets the entire student list for a single university
    params: url(string) - student list url of the university
            session_request(session obejct) - session with login posted
            admit(string) - admit/reject status
    return: student_detail(dataframe) - containg all the student list for a single university
    """
    page_counter = 1
    student_detail = pd.DataFrame()
    url = url + "?page="
    target_url = url + str(page_counter)
    while(page_counter <= 50):
        
        try:
            page_soup = open_url_2_soup(target_url,session_request)
        except HTTPError:
            print(target_url)
            print("exception raised")
            page_counter = 1
            break
            
        else:
            print(target_url)
            target_url = url + str(page_counter)
            std_detail_page = getStdList(page_soup,admit)
            student_detail = pd.concat([student_detail,std_detail_page])
            time.sleep(10)
            page_counter += 1
        
    return student_detail

In [ ]:
def generateStudentList():
    """
    generate applied and rejected student detail list for all universities
    params: None
    returns: None
    """
    token, session_request = getAuthSession()
#     session_request = None
#     print(token)
    student_detail_df = pd.read_csv("universityDetailList.csv", header=1)
    header = False
    for index,row in student_detail_df.iterrows():
        if(index in range(250,301)):
            print(index)
#           if(index == 100):
            student_admit = getStudentDetailList(row['admitList'], session_request, 'admit')
#             print(student_admit)
            student_admit.to_csv('studentList1.csv', mode = 'a', encoding='utf-8',header = header)
            header = False
            student_reject = getStudentDetailList(row['rejectList'], session_request, 'reject')
            student_reject.to_csv('studentList1.csv', mode='a', encoding = 'utf-8', header = header)
#             time.sleep(120)
       

In [ ]:
generateStudentList()

In [ ]:
import requests
url = 'https://yocket.in/applications-admits-rejects/new-york-institute-of-technology-manhattan/2'
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}

response = requests.get(url,proxies=proxies, headers = headers,verify = False, allow_redirects = False)
print(response.status_code)

    

In [15]:
student = pd.read_csv('studentList.csv',header=1)
student1 = pd.read_csv('studentList1.csv',header=None, names = ['Nt req','CGPA','CourseName','ENG_TEST','GRE','Name','WorkExp','admit/reject','college'])
print(student.shape, student1.shape)
concat = [student,student1]
concat_df = pd.concat(concat)
print("concated final shape",concat_df.shape)
student.tail(3)

(24559, 9) (28084, 9)
concated final shape (52643, 10)


,Unnamed: 0,CGPA,CourseName,ENG_TEST,GRE,Name,WorkExp,admit/reject,college
24556,8.0,\r\n9.29 CGPA,"\r\nUniversity of California, Santa Barbara Co...",\r\n108,\r\n313,Srushti Sb,\r\n20 months,admit,"University of California, Santa Barbara"
24557,9.0,\r\n8.8 CGPA,"\r\nUniversity of California, Santa Barbara El...",\r\n109,\r\n325,Tajuddin Manhar,\r\nNA,admit,"University of California, Santa Barbara"
24558,10.0,\r\n9.54 CGPA,"\r\nUniversity of California, Santa Barbara El...",\r\n109,\r\n329,Prathi,\r\n30 months,admit,"University of California, Santa Barbara"


In [16]:
concat_df.head(3)

,CGPA,CourseName,ENG_TEST,GRE,Name,Nt req,Unnamed: 0,WorkExp,admit/reject,college
0,\r\n67 %,\r\nNortheastern University Information System...,\r\n96,\r\n310,Achira,NaN,0.0,\r\n36 months,admit,Northeastern University
1,\r\n65 %,\r\nNortheastern University Information System...,\r\n96,\r\n310,achira shah,NaN,1.0,\r\n24 months,admit,Northeastern University
2,\r\n7.85 CGPA,\r\nNortheastern University Engineering Manage...,\r\n100,\r\n304,Deep Vira,NaN,2.0,\r\n1 month,admit,Northeastern University


In [17]:
student1.head(3)

,Nt req,CGPA,CourseName,ENG_TEST,GRE,Name,WorkExp,admit/reject,college
0,0,\r\n7.85 CGPA,"\r\nNorth Carolina State University, Raleigh M...",\r\n100,\r\n 304,Deep Vira,\r\n1 month,reject,"North Carolina State University, Raleigh"
1,1,\r\n69.6 %,"\r\nNorth Carolina State University, Raleigh A...",\r\n7,\r\n306,yash mehta,\r\n6 months,reject,"North Carolina State University, Raleigh"
2,2,\r\n8.85 CGPA,"\r\nNorth Carolina State University, Raleigh M...",\r\n118,\r\n329,Srinidhi Suresh,\r\nNA,reject,"North Carolina State University, Raleigh"
